<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions_close.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO: You broke it. Go back and fix it, try to understand the lstm params. 

In [83]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists
from decimal import *
#pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [122]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()

coin_base = False
ku_coin = True
COINBASE_REST_API = 'https://api.pro.coinbase.com' 
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"
trend_periods = [7, 15]
trend_labels = ["7Trend", "15Trend"]
feature_columns = ["Close", "Volume"] + trend_labels
num_features = len(feature_columns)

data_path = '/content/drive/My Drive/output.csv'
model_path = "/content/drive/My Drive/model_long_trends.h5"
old_model_path = "/content/drive/My Drive/model_ohlc.h5"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_trend_slope(df, durations):
  for duration in durations:
    df[str(duration) + 'Close'] = df['Close'].shift(duration)
    df[str(duration) + 'Date'] = df['Date'].shift(duration)
    df[str(duration) + 'Trend'] = (df['Close'] - df[str(duration) + 'Close']) / duration  
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)
  
  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we 
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history
  
def gran_to_string(granularity):
  #todo implement this actually
  return "1day"

def get_coin_data_frames(time, product, granularity='86400'):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = df_btc_history.drop(columns={"high", "low", "open"})
  df_btc_history = append_price_dif_(df_btc_history)
  df_btc_history = append_trend_slope(df_btc_history, trend_periods)
  df_btc_features = df_btc_history[feature_columns]
  df_history_scaled = sc.fit_transform(df_btc_features)
  
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()
  
  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(product, bars):
  [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, product)
  predicted = model.predict(npa_scaled).flatten()
  
  #convert to data frames that have the correct shape for being unscaled
  df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume"])
  
  # I want to believe that scaling happens on a per column basis, we only care about
  # price here so we will dummy out volume and trend and use the scaler on it
  # this kinda sucks, if we add features we'll need to add them here for unscaling
  df_temp = pd.DataFrame(predicted, columns = ["Close"])
  df_temp = zero_irrelevant_columns(df_temp)
  
  # unscale them both
  df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
  df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
  
  # add predicted
  df_trade["Predicted"] = df_temp["Close"]
  df_trade = df_trade.tail(1)

  # add the product, derive a move and percent
  df_trade["Product"] = product;
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"] 
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade

def zero_irrelevant_columns(df):
  df["Volume"] = 0
  for label in trend_labels:
    df[label] = 0
  return df

def get_training_set_for(ticker):
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, ticker)
  target_df = append_price_dif(target_df)
  target_df = append_trend_slope(target_df, trend_periods)
  features = target_df[trend_labels + ["Close", "Volume", "Target"]]
  scaled_features = scale_data(features)
  X = []
  y = []
  for i in range(0, len(target_df)):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]
  
def train_model(model, X, y):

  # One day we might need test, but for now we don't we can use another
  # time series, we have so many
  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2
  )
  return [model, history]

#pull training data 
all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/Colab Notebooks/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_volume.csv"))

#Library

# Get or Train a Model

In [123]:
target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, "sp500")
target_df = append_price_dif(target_df)
target_df = append_trend_slope(target_df, trend_periods)
features = target_df[trend_labels + ["Close", "Volume", "Target"]]
features

,7Trend,15Trend,Close,Volume,Target
15,2.691424,3.293335,1344.900024,4608550000,1344.329956
16,3.699986,3.682666,1344.329956,3379700000,1347.050049
17,4.388585,3.558667,1347.050049,3742460000,1349.959961
18,5.278564,2.794661,1349.959961,4096730000,1351.949951
19,5.648560,2.496663,1351.949951,4209890000,1342.640015
...,...,...,...,...,...
2153,12.982875,7.265999,3306.510010,4621670000,3327.770020
2154,12.622873,6.747331,3327.770020,4732220000,3349.159912
2155,18.674282,8.905990,3349.159912,4267490000,3351.280029
2156,13.262870,8.436670,3351.280029,4104860000,3360.469971


In [124]:
model = None
file_exists = exists(model_path)

if file_exists:
  print("hello")
  model = keras.models.load_model(model_path)
else:
  print("nope")
  [scaled_features, X, y] = get_training_set_for("sp500")  
  model = build_model(num_features, 1)
  [model, history] = train_model(model, X, y)


nope
Epoch 1/20
54/54 [==============================] - 15s 122ms/step - loss: 0.0465 - val_loss: 0.1285
Epoch 2/20
54/54 [==============================] - 3s 46ms/step - loss: 0.0295 - val_loss: 0.0799
Epoch 3/20
54/54 [==============================] - 2s 38ms/step - loss: 0.0115 - val_loss: 0.0459
Epoch 4/20
54/54 [==============================] - 2s 37ms/step - loss: 0.0065 - val_loss: 0.0242
Epoch 5/20
54/54 [==============================] - 2s 37ms/step - loss: 0.0049 - val_loss: 0.0294
Epoch 6/20
54/54 [==============================] - 2s 38ms/step - loss: 0.0041 - val_loss: 0.0214
Epoch 7/20
54/54 [==============================] - 2s 38ms/step - loss: 0.0037 - val_loss: 0.0152
Epoch 8/20
54/54 [==============================] - 2s 39ms/step - loss: 0.0034 - val_loss: 0.0163
Epoch 9/20
54/54 [==============================] - 2s 39ms/step - loss: 0.0028 - val_loss: 0.0131
Epoch 10/20
54/54 [==============================] - 2s 38ms/step - loss: 0.0028 - val_loss: 0.0131
Ep

In [114]:
# additional training?
if 1:
  [scaled_features, X, y] = get_training_set_for("TSLA")  
  [model, history] = train_model(model, X, y)

Epoch 1/20
51/51 [==============================] - 3s 53ms/step - loss: 5.9667e-04 - val_loss: 0.0022
Epoch 2/20
51/51 [==============================] - 3s 51ms/step - loss: 4.2934e-04 - val_loss: 0.0024
Epoch 3/20
51/51 [==============================] - 3s 52ms/step - loss: 3.1582e-04 - val_loss: 0.0017
Epoch 4/20
51/51 [==============================] - 3s 53ms/step - loss: 2.7947e-04 - val_loss: 0.0018
Epoch 5/20
51/51 [==============================] - 3s 51ms/step - loss: 2.1316e-04 - val_loss: 0.0022
Epoch 6/20
51/51 [==============================] - 3s 52ms/step - loss: 1.8914e-04 - val_loss: 0.0018
Epoch 7/20
51/51 [==============================] - 3s 54ms/step - loss: 1.7657e-04 - val_loss: 0.0016
Epoch 8/20
51/51 [==============================] - 3s 51ms/step - loss: 1.5610e-04 - val_loss: 0.0014
Epoch 9/20
51/51 [==============================] - 3s 51ms/step - loss: 1.2128e-04 - val_loss: 0.0015
Epoch 10/20
51/51 [==============================] - 3s 51ms/step - loss:

# Visualize and Backtest

In [125]:
budget = 5000
[btc_history, df_btc_features, npa_scaled, df_raw] = get_coin_data_frames(180, "BTC-USDT")
predicted = model.predict(npa_scaled).flatten()[::-1] 
df_temp = pd.DataFrame(predicted, columns = ["Close"])
df_temp = zero_irrelevant_columns(df_temp)
df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = feature_columns)
btc_history["Predicted"] = df_temp["Close"]
[df_profit, profit] = build_profit_estimate(predicted, btc_history)
interactive_plot(btc_history[["Date","Close", "Predicted"]], "Actual vs Predicted")
print("Profit:", profit)


6/6 [==============================] - 2s 12ms/step


Profit: -860.3540479165064


# Which coins are most profitable based on the above trading signals?

In [ ]:
# download all known products and check who has the highest profit in 90 days
"""
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USD')]
df_products = df_products[df_products.trading_disabled == False]
df_products = df_products[df_products.cancel_only == False]
df_profit = pd.DataFrame();
df_profit["Product"] = [];
df_profit["Profit"] = [];
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)
    predicted = model.predict(df_scaled.flatten())
    [df_chart, profit] = build_profit_estimate(predicted, df_full)
    df_profit.loc[len(df_profit.index)] = [row.id, profit] 
  except Exception as inst:
    print("Error: ", inst)
  time.sleep(1)

df_profit
"""

'\n# Fetch the top 10 and see if they predict up\ndf_products = get_all_products()\ndf_products = df_products[df_products.id.str.endswith(\'USD\')]\ndf_products = df_products[df_products.trading_disabled == False]\ndf_products = df_products[df_products.cancel_only == False]\ndf_profit = pd.DataFrame();\ndf_profit["Product"] = [];\ndf_profit["Profit"] = [];\nfor index, row in df_products.iterrows():\n  try:\n    print("fetching: ", row.id)\n    [df_full, df_features, df_scaled] = get_coin_data_frames_test(90, row.id)\n    predicted = model.predict(df_scaled.flatten())\n    [df_chart, profit] = build_profit_estimate(predicted, df_full)\n    df_profit.loc[len(df_profit.index)] = [row.id, profit] \n  except Exception as inst:\n    print("Error: ", inst)\n  time.sleep(1)\n\ndf_profit\n'

# What has a buy indicator for tomorrow?

In [ ]:
predict_trade("DOGE3L-USDT", 90)

3/3 [==============================] - 0s 15ms/step


,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.000472,2.254780e+08,-0.000036,0.000526,DOGE3L-USDT,0.000054,11.380349,0.113803,2.0,40.0,50.0,26.450872,529.017438,1088.034876


In [ ]:
predict_trade("FCON-USDT", 90)

3/3 [==============================] - 0s 9ms/step


,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.000045,1.721042e+09,-0.000009,0.000051,FCON-USDT,0.000006,12.705889,0.127059,2.0,40.0,50.0,29.764723,595.294454,1220.588907


In [ ]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
bars = 90
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    
    df_trade = predict_trade(row.id, bars)
   
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(0.5)
df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 




fetching:  NKN-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  GEM-USDT
Error:  Requested:90 bars  but only got:78
fetching:  CUSD-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  LTC3L-USDT
3/3 [==============================] - 0s 13ms/step
fetching:  OAS-USDT
Error:  Requested:90 bars  but only got:16
fetching:  KNC-USDT
3/3 [==============================] - 0s 9ms/step
fetching:  LYM-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  HAI-USDT
3/3 [==============================] - 0s 9ms/step
fetching:  MITX-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  PDEX-USDT
3/3 [==============================] - 0s 12ms/step
fetching:  FLAME-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  EPX-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  AOG-USDT
Error:  Requested:90 bars  but only got:89
fetching:  ATOM3L-USDT
3/3 [==============================] - 0s 9ms/step
fetching:  A

In [ ]:
df_shorts

,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.083305,8.374552e+04,0.000194,0.082526,NKN-USDT,-0.000779,-0.935604,-0.009356,2.0,40.0,50.0,-4.339010,-86.780207,-143.560414
74,0.996800,2.301907e+04,0.000033,0.994825,CUSD-USDT,-0.001975,-0.198125,-0.001981,2.0,40.0,50.0,-2.495314,-49.906273,-69.812545
74,0.001392,2.880934e+07,-0.000045,0.001387,LTC3L-USDT,-0.000005,-0.374473,-0.003745,2.0,40.0,50.0,-2.936183,-58.723663,-87.447326
74,0.522000,1.845736e+04,-0.006867,0.520911,KNC-USDT,-0.001089,-0.208630,-0.002086,2.0,40.0,50.0,-2.521575,-50.431507,-70.863014
74,0.001193,1.712094e+06,-0.000002,0.001171,LYM-USDT,-0.000022,-1.843701,-0.018437,2.0,40.0,50.0,-6.609253,-132.185056,-234.370113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.022930,8.060674e+05,-0.000053,0.022927,STORE-USDT,-0.000003,-0.012214,-0.000122,2.0,40.0,50.0,-2.030535,-40.610691,-51.221383
74,0.459900,4.895358e+05,-0.002260,0.454928,FLUX-USDT,-0.004972,-1.081017,-0.010810,2.0,40.0,50.0,-4.702541,-94.050828,-158.101657
74,0.006078,1.368320e+06,0.000113,0.006005,CERE-USDT,-0.000073,-1.196449,-0.011964,2.0,40.0,50.0,-4.991123,-99.822458,-169.644917
74,0.000990,4.626607e+07,-0.000019,0.000978,FRR-USDT,-0.000012,-1.236238,-0.012362,2.0,40.0,50.0,-5.090594,-101.811876,-173.623751


In [ ]:
df_buys

,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
74,0.002934,2.681275e+07,-0.000053,0.002970,MITX-USDT,3.633959e-05,1.238568,0.012386,2.0,40.0,50.0,1.096421,21.928414,73.856828
74,0.020300,7.692874e+06,-0.000172,0.020569,FLAME-USDT,2.686402e-04,1.323351,0.013234,2.0,40.0,50.0,1.308377,26.167540,82.335079
74,0.073500,3.679475e+04,-0.000376,0.073751,YLD-USDT,2.508677e-04,0.341317,0.003413,2.0,40.0,50.0,-1.146709,-22.934171,-15.868341
74,0.222000,1.480038e+05,-0.005600,0.224207,CWS-USDT,2.206556e-03,0.993944,0.009939,2.0,40.0,50.0,0.484860,9.697197,49.394393
74,1.071000,9.703112e+03,-0.012800,1.073180,ALICE-USDT,2.180058e-03,0.203553,0.002036,2.0,40.0,50.0,-1.491116,-29.822326,-29.644651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.005050,5.860800e+07,-0.000050,0.005051,UPO-USDT,6.487185e-07,0.012846,0.000128,2.0,40.0,50.0,-1.967885,-39.357704,-48.715409
74,0.001581,3.837938e+07,-0.000035,0.001602,2CRZ-USDT,2.087899e-05,1.320619,0.013206,2.0,40.0,50.0,1.301548,26.030951,82.061903
74,0.413500,2.928138e+05,-0.006220,0.415075,RNDR-USDT,1.574617e-03,0.380802,0.003808,2.0,40.0,50.0,-1.047994,-20.959889,-11.919778
74,0.003010,8.298926e+07,-0.000049,0.003026,DMTR-USDT,1.571014e-05,0.521932,0.005219,2.0,40.0,50.0,-0.695171,-13.903425,2.193151


In [ ]:
with open(data_path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [ ]:
model.save(model_path)